In [1]:
import xml.etree.ElementTree as ET
import csv


# Pour que Python comprenne les éléments dont on parlera,
# il faut lui donner la déclaration TEI, mais comme c'est
# le NS de base, pas besoin de lui donner un préfixe.
ET.register_namespace('', "http://www.tei-c.org/ns/1.0")

In [19]:
def get_w_txt(word):
    
    amorig = ''
    expmod = ''
    
    # S'il y a du texte avant le premier enfant, on l'ajoute.
    if word.text:
        
        amorig += word.text
        expmod += word.text
    
    # On boucle sur les enfants du <w> actuel.
    for item in word:
        
        checklist = [
            '{http://www.tei-c.org/ns/1.0}height',
            '{http://www.tei-c.org/ns/1.0}supplied',
            '{http://www.tei-c.org/ns/1.0}c',
            '{http://www.tei-c.org/ns/1.0}hi'
        ]
    
        if item.tag in checklist:
            amorig += item.text
            expmod += item.text
            if item.tail:
                amorig += item.tail
                expmod += item.tail
            
        elif item.tag == '{http://www.tei-c.org/ns/1.0}lb':
            if item.tail:
                amorig += item.tail
                expmod += item.tail
            
        elif item.tag == '{http://www.tei-c.org/ns/1.0}choice':
            
            for subitem in item.findall('./*'):
                if subitem.tag == '{http://www.tei-c.org/ns/1.0}am' or subitem.tag == '{http://www.tei-c.org/ns/1.0}orig':
                    amorig += subitem.text
                            
                elif subitem.tag == '{http://www.tei-c.org/ns/1.0}expan' or subitem.tag == '{http://www.tei-c.org/ns/1.0}reg':
                    try:
                        expmod += subitem.text
                    except:
                        "ALERTE !!"
                
            if item.tail:
                amorig += item.tail
                expmod += item.tail
                
                
        elif item.tag == '{http://www.tei-c.org/ns/1.0}add':
            # On refait tous les tests.
            for subitem in item:
    
            # Si l'enfant est un <height>, on récupère son texte.
                if subitem.tag in checklist:
                    amorig += subitem.text
                    expmod += subitem.text
                    if item.tail:
                        amorig += subitem.tail
                        expmod += subitem.tail

                elif subitem.tag == '{http://www.tei-c.org/ns/1.0}lb':
                    if subitem.tail:
                        amorig += subitem.tail
                        expmod += subitem.tail

                # Si l'enfant est un <choice>, on récupère le texte de son
                # second enfant et on vérifie s'il y a du texte après le <choice>.
                elif subitem.tag == '{http://www.tei-c.org/ns/1.0}choice':
            
                    for subsub in subitem.findall('./*'):
                        if subsub.tag == '{http://www.tei-c.org/ns/1.0}am' or subsub.tag == '{http://www.tei-c.org/ns/1.0}orig':
                            amorig += subsub.text
                            
                        elif subsub.tag == '{http://www.tei-c.org/ns/1.0}expan' or subsub.tag == '{http://www.tei-c.org/ns/1.0}reg':
                            expmod += subsub.text

                            if subitem.tail:
                                amorig += subitem.tail
                                expmod += subitem.tail
    
    if amorig != expmod:
        texte = [amorig, expmod]
    else:
        texte = ['', expmod]
            
    return texte

In [24]:
#Revue le 15 avril. Options pour fichier déjà lemmatisé ajoutées.

def export_tokens_to_csv(chemin_entree, chemin_sortie):
    
    """
    Fonction récupérant les éléments <w> d'un fichier XML-TEI
    avec leurs @n pour les exporter dans un fichier CSV à
    deux colonnes, l'une pour le numéro, l'autre pour le texte.
    Si l'élément <w> comporte des enfants <height> pour les initiales
    hautes, ou <choice> pour des modernisations/abréviations,
    la fonction compilera une version complète, résolue et modernisée
    pour le logiiel Analog.
    
    :param chemin_entree: Le chemin local vers un fichier en XML-TEI
        dont on souhaite extraire les tokens pour Analog.
    :param chemin_sortie: L'emplacement local dans lequel on souhaite
        écrire le fichier CSV à donner à Analog.    
    """
   
    l_count = 0
    
    lbinitlist = []
    winitdict = {}
    
    # Le dictionnaire dans lequel seront stockés les tokens avant écriture en CSV.
    colonnes = [
        'n° de ligne',
        'nature',
        'n°/id',
        'forme "diplo"',
        'forme modernisée',
        'forme corrigée',
        'lemme',
        'pos',
        'à scinder',
        'à fusionner avec w n°',
        'à corriger',
        'corrigé'
    ]
    
    with open(chemin_entree) as infile:
    
        # On va chercher le fichier XML-TEI et on le lit.
        tree = ET.parse(infile)
        root = tree.getroot()
        
        for parent in root.findall('.//*[{http://www.tei-c.org/ns/1.0}w]'):
            for child in parent.findall('./*'):
                if child.tag == '{http://www.tei-c.org/ns/1.0}w':
                    lbinitlist.append(child.get('n'))
                elif child.tag == '{http://www.tei-c.org/ns/1.0}lb':
                    lbinitlist.append(child.get('facs'))
                    
        for word in root.findall('.//{http://www.tei-c.org/ns/1.0}w'):
            texte = ''
            numero = str(word.get('n'))
            lemmes = str(word.get('lemma'))
            pos = str(word.get('pos'))
            
            winitdict[numero] = {'lemma':lemmes, 'pos':pos}
            
            if word.find('{http://www.tei-c.org/ns/1.0}lb'):
                winitdict[numero]['lb'] = word.find('{http://www.tei-c.org/ns/1.0}lb').get('facs')
            else:
                wIndex = lbinitlist.index(numero)
                try:
                    nextIndex = lbinitlist[wIndex+1]
                except:
                    nextIndex = ''
                if "_"in nextIndex:
                    winitdict[numero]['lb'] = nextIndex
                else:
                    winitdict[numero]['lb'] = 'None'
            
            if word.find('./*') == None :
                winitdict[numero]['original'] = ''
                winitdict[numero]['modernisé'] = word.text

            else:
                winitdict[numero]['original'] = get_w_txt(word)[0]
                winitdict[numero]['modernisé'] = get_w_txt(word)[1]

    # On ouvre le CSV de sortie en mode "écriture", on y écrit le nom des colonnes.
    with open(chemin_sortie, 'w') as csv_file:
        csv_contenu = csv.DictWriter(csv_file, fieldnames = colonnes)
        csv_contenu.writeheader()

        # On boucle sur les éléments <w> du XML, dans l'ordre du fichier.
        # for word in root.findall('.//{http://www.tei-c.org/ns/1.0}w'):
        for word in winitdict.keys():
            
            dicolocal = winitdict[word]

            l_count += 1
                
            if '|' in dicolocal['pos'] or dicolocal['pos'] == 'Inconnu':
                a_corriger = 'X'
            else:
                a_corriger = ''

            
            if dicolocal['lb'] != 'None':
                csv_contenu.writerow({
                    'n° de ligne' : str(l_count),
                    'nature' : 'saut de ligne',
                    'n°/id' : dicolocal['lb'],
                    'forme "diplo"' :'',
                    'forme modernisée':'',
                    'forme corrigée':'',
                    'lemme' :'',
                    'pos' :'',
                    'à scinder' :'',
                    'à fusionner avec w n°' :'',
                    'à corriger' :'',
                    'corrigé' :''
                })

                l_count += 1

                csv_contenu.writerow({
                    'n° de ligne' : str(l_count),
                    'nature' : 'w',
                    'n°/id' : word,
                    'forme "diplo"' : dicolocal['original'],
                    'forme modernisée' : dicolocal['modernisé'],
                    'forme corrigée':'',
                    'lemme' : dicolocal['lemma'],
                    'pos' : dicolocal['pos'],
                    'à scinder' :'',
                    'à fusionner avec w n°' :'',
                    'à corriger' : a_corriger,
                    'corrigé' :''
                })

            else:

                csv_contenu.writerow({
                    'n° de ligne' : str(l_count),
                    'nature' : 'w',
                    'n°/id' : word,
                    'forme "diplo"' : dicolocal['original'],
                    'forme modernisée' : dicolocal['modernisé'],
                    'forme corrigée':'',
                    'lemme' : dicolocal['lemma'],
                    'pos' : dicolocal['pos'],
                    'à scinder' :'',
                    'à fusionner avec w n°' :'',
                    'à corriger' : a_corriger,
                    'corrigé' :''
                })

In [ ]:
temoins = ['basnage','berault','gc','institutions','merville','morisse','pesnelle','rouille','ruines','tac','terrien']

for temoin in temoins:
    export_tokens_to_csv('/home/erminea/Documents/CONDE/editions/base-version/' + temoin + '_base.xml',
                 temoin + '_tableau_pour_corrections.csv')
    print(temoin + " : terminé")

basnage : terminé
